# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686558


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:06<01:28,  3.05s/it]

Rendering models:  10%|▉         | 3/31 [00:07<01:11,  2.56s/it]

Rendering models:  13%|█▎        | 4/31 [00:07<00:50,  1.87s/it]

Rendering models:  16%|█▌        | 5/31 [00:08<00:37,  1.45s/it]

...

Rendering models:  90%|█████████ | 28/31 [00:13<00:00,  6.03it/s]

Rendering models:  94%|█████████▎| 29/31 [00:14<00:00,  4.53it/s]

Rendering models:  97%|█████████▋| 30/31 [00:14<00:00,  3.34it/s]

Rendering models: 100%|██████████| 31/31 [00:15<00:00,  3.04it/s]

2022_mayaw                            0.002821
not-logged-in-1737c3b79f1a3fbc13ec    0.001669
not-logged-in-6f1c0c5f52883219bbbd    0.003043
Slartibartfast11                      0.024405
CThomas                               0.001021
cschwefl                              0.000612
shocko61                              0.000709
shocko61                              0.000526
Roquwat                               0.001803
not-logged-in-6c3c85931d83a1164a0d    0.004134
MakenzieHoward                        0.033553
sn346808                              0.032242
not-logged-in-f7e2260ed4a9517a3e4a    0.005205
not-logged-in-ffc5c906399855f5ef0f    0.000821
kaitybug                              0.001508
puic                                  0.000630
H_Robinson                            0.003900
VirK_QSO                              0.001077
bnamumba                              0.000688
not-logged-in-e23c11d8538d9848a6a5    0.002963
ranyel.a.e.pasimio                    0.000537
Flavius44    

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())